In [1]:
%load_ext nb_black
%load_ext lab_black

<IPython.core.display.Javascript object>

In [2]:
import numpy as np
import pandas as pd
import tqdm

<IPython.core.display.Javascript object>

In [3]:
"""
train data 또는 test data 중 feature를 추가하고 싶은 데이터를 불러오면 됩니다.!
(train과 test data를 합친 data를 불러와도 됩니다!)
"""
# TODO : 원하는 데이터의 경로를 설정해주세요
raw_data = pd.read_csv("/opt/ml/input/data/FE_total_data.csv")

user_id = raw_data["userID"].unique()  # 유저 아이디 저장

<IPython.core.display.Javascript object>

In [4]:
raw_data  # 원본 확인

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1
...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1


<IPython.core.display.Javascript object>

In [5]:
##-------------------대분류 Feature 추가--------------------------
raw_data["bigClass"] = raw_data["testId"].str[2]

<IPython.core.display.Javascript object>

In [6]:
raw_data  # 대분류 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,bigClass
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,6
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,6
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,6
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,6
...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,3
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,4
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,4


<IPython.core.display.Javascript object>

In [7]:
"""
acc_by_big_category
    - key : 유저 아이디
    - value : 해당 유저의 대분류별 정답률 (인덱스가 대분류를 나타낸다. ex. index=1 -> 대분류 1)
    - default value : 기본적으로 0.5라는 값이 들어간다. 
"""
acc_by_big_category = dict()
for user in user_id:
    acc_by_big_category[user] = [0.5] * 10

<IPython.core.display.Javascript object>

In [12]:
for user in tqdm.tqdm(user_id):  # 모든 유저에 대해
    data = raw_data[raw_data["userID"] == user]  # 각 유저에 대한 행만 추출하여 data에 저장
    bc_l = data["bigClass"].unique()  # 행들에 있는 고유한 대분류 추출 ex) [6, 2, 8]

    for big_class in bc_l:  # 유저가 풀어본 각 대분류에 대해
        big_class_int = int(big_class)
        user_class_interaction = data[data["bigClass"] == big_class]

        big_correct = 0
        big_total = len(user_class_interaction)

        for is_correct in user_class_interaction["answerCode"]:
            if is_correct == 1:
                big_correct += 1

        acc_by_big_category[user][big_class_int] = (
            big_correct / big_total
        )  # 전체 데이터를 순회하면서 알게된 대분류 정답률을 저장

100%|██████████| 7442/7442 [00:28<00:00, 261.49it/s]


<IPython.core.display.Javascript object>

In [14]:
"""
acc_by_big_category_list != acc_by_big_category
: 해당 리스트는 feature를 추가하기 전에 feature 값들을 일렬로 저장하기 위해 만든 리스트입니다.
"""
acc_by_big_category_list = list()

for u, bc in tqdm.tqdm(
    zip(raw_data["userID"], raw_data["bigClass"]), total=len(raw_data["userID"])
):  # 모든 행(instance)에 대해
    acc = acc_by_big_category[u][
        int(bc)
    ]  # 해당 유저의 해당 대분류에 대한 정답률을 위에서 정의한 dictionary에서 찾아서
    acc_by_big_category_list.append(acc)  # 리스트에 저장한다.

raw_data["bigClassAcc"] = acc_by_big_category_list  # 리스트에 bigClassAcc feature 추가

100%|██████████| 2526700/2526700 [00:02<00:00, 872617.89it/s]


<IPython.core.display.Javascript object>

In [15]:
raw_data  # 대분류, 대분류별 정답률 추가 이후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,bigClass,bigClassAcc
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,0.791908
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,6,0.791908
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,6,0.791908
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,6,0.791908
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,6,0.791908
...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,3,0.200000
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,1.000000
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,4,1.000000
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,4,1.000000


<IPython.core.display.Javascript object>

In [17]:
"""
acc_cate_list : 각 대분류 정답률을 카테고리로 만들어 저장하려고 한 리스트. 
ex. 원래 값 [0.04, 0.79, 0.43, 0.99] -> 곱하기 10 : [0.4, 7.9, 4.3, 9.9] -> round : [0, 8, 4, 10] -> 더하기 1 : [1, 9, 5, 11]
"""
acc_cate_list = list()
for big_class_acc in raw_data["bigClassAcc"]:
    acc = big_class_acc * 10
    acc = round(acc) + 1
    acc_cate_list.append(acc)

raw_data["bigClassAccCate"] = acc_cate_list  # 리스트로 저장

<IPython.core.display.Javascript object>

In [18]:
raw_data  # 대분류, 대분류별 정답률, 대분류별 정답률 카테고리 추가 후

,userID,assessmentItemID,testId,answerCode,Timestamp,KnowledgeTag,dataset,bigClass,bigClassAcc,bigClassAccCate
0,0,A060001001,A060000001,1,2020-03-24 00:17:11,7224,1,6,0.791908,9
1,0,A060001002,A060000001,1,2020-03-24 00:17:14,7225,1,6,0.791908,9
2,0,A060001003,A060000001,1,2020-03-24 00:17:22,7225,1,6,0.791908,9
3,0,A060001004,A060000001,1,2020-03-24 00:17:29,7225,1,6,0.791908,9
4,0,A060001005,A060000001,1,2020-03-24 00:17:36,7225,1,6,0.791908,9
...,...,...,...,...,...,...,...,...,...,...
2526695,7441,A030071005,A030000071,0,2020-06-05 06:50:21,438,1,3,0.200000,3
2526696,7441,A040165001,A040000165,1,2020-08-21 01:06:39,8836,1,4,1.000000,11
2526697,7441,A040165002,A040000165,1,2020-08-21 01:06:50,8836,1,4,1.000000,11
2526698,7441,A040165003,A040000165,1,2020-08-21 01:07:36,8836,1,4,1.000000,11


<IPython.core.display.Javascript object>

In [19]:
raw_data = raw_data.sort_values(by=["userID", "Timestamp"])

<IPython.core.display.Javascript object>

In [20]:
# TODO : 이름과 경로 원하는 데로 변경하기
import os

os.makedirs("/opt/ml/input/data/FE", exist_ok=True)
raw_data.to_csv(
    "/opt/ml/input/data/FE/bigClassAcc.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "bigClassAcc",
    ],
    index=False,
)  # 최종 결과 csv로 반환
raw_data.to_csv(
    "/opt/ml/input/data/FE/bigClassAccCate.csv",
    columns=[
        "userID",
        "assessmentItemID",
        "testId",
        "answerCode",
        "Timestamp",
        "KnowledgeTag",
        "dataset",
        "bigClassAccCate",
    ],
    index=False,
)  # 최종 결과 csv로 반환

<IPython.core.display.Javascript object>